In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
xx = QQ['xx'].0
Q.<i_rad3> = NumberField(xx^2+3)

In [ ]:
# Define cubic SE9, find and classify the lines and compute the 45 tritangent planes

In [3]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [4]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
%time SE9 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({e : i_rad3*c})

CPU times: user 5.09 s, sys: 4.52 ms, total: 5.09 s
Wall time: 5.1 s


In [ ]:
# Find simmetries

In [5]:
%time adm_SE9 = SE9.find_admissible_projectivities()
len(adm_SE9)

CPU times: user 3.9 s, sys: 152 ms, total: 4.05 s
Wall time: 51.7 s


1296

In [6]:
%time simm_SE9 = SE9.find_simmetries(adm_SE9)
len(simm_SE9)

CPU times: user 2.16 s, sys: 179 ms, total: 2.34 s
Wall time: 6.76 s


54

In [ ]:
# Study how the simmetries permute the Eckardt points

In [7]:
Eck_points_perms = []
for proj in simm_SE9:
    Eck_points_perms.append(Permutation(SE9.apply_proj_to_eck(proj)).to_permutation_group_element())
Eck_points_perms_group = PermutationGroup(Eck_points_perms)
Eck_points_perms_group.order(), Eck_points_perms_group.structure_description()

(18, '(C3 x C3) : C2')

In [ ]:
# Study how the simmetries permute the 27 lines

In [8]:
lines_perms = []
for simm in simm_SE9:
    lines_perms.append(Permutation(SE9.apply_proj_to_lines(simm)).to_permutation_group_element())
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(54, '((C3 x C3) : C3) : C2')

In [ ]:
# There are twelve lines through 3 Eckardt points, and they are all coplanar

In [ ]:
lines_three_eck = []
for i in range(7):
    for j in range(i+1, 8):
        for k in range(j+1, 9):
            point1 = SE9.eckardt_points[i]
            point2 = SE9.eckardt_points[j]
            point3 = SE9.eckardt_points[k]   
            #if matrix([point1, point2, point3]).minors(3) == [0,0,0,0]:
                #print(SE9.eckardt_points_labels[i], SE9.eckardt_points_labels[j], SE9.eckardt_points_labels[k])
            line12_pl = Point(matrix([point1, point2]).minors(2))
            line23_pl = Point(matrix([point2, point3]).minors(2))
            if line12_pl == line23_pl and line12_pl not in [line.plucker for line in lines_three_eck]:
                planes = get_two_planes_containing_line(line12_pl)
                lines_three_eck.append(Line(planes,[point1, point2], line12_pl))

In [12]:
bool([[line1.are_incident(line2) for line2 in lines_three_eck] for line1 in lines_three_eck])

True

In [11]:
lines_three_eck[0].get_plane_containing_another_incident_line(lines_three_eck[1])

x + (-1/4*alpha - 1/4)*y + (1/4*alpha + 1/4)*z

In [ ]:
# To do, study how the 12 lines are permuted